<a href="https://colab.research.google.com/github/O-Connor-Lab/automation/blob/main/one_plate_ELISA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
def log5pl(x, A, B, C, D, E):
    return((A-D)/((1.0+((x/C)**B))** E) + D)

In [ ]:
def rev_log5pl(y, A, B, C, D, E):
    return C * (((((A-D)/(y-D)) ** (1/E)) - 1) ** (1/B))

In [ ]:
metadata = pd.read_excel('metadata_ELISA.xlsx')

In [ ]:
plates = 1

In [ ]:
raw_data = {}

file = open("test_ELISA_encoded.txt", "r")
lines = file.readlines()
del lines[0:3] #deletes the top three lines from the files

for i in range(plates):

    plate = lines[i * 12: i * 12 + 10] # extracts the 10 lines for each plate
    plate_name = plate[0].split('\t')[1] #pulls out the plate number
    plate_values = [] #initializes an empty list

    for j in range(2,10): #goes through each line of the plate
        row = plate[j].split('\t')
        row.remove('\n') #removes the \n from the list
        row[:] = [x for x in row if x] #
        plate_values.append(row)
    del plate_values[0][0]
    plate_values = pd.DataFrame(plate_values, columns=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
                               index = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']).astype(float)
    raw_data.update({plate_name : plate_values})

In [ ]:
#bkgrnd_subtracted = raw_data.copy()

#for keys in bkgrnd_subtracted.keys():
 #   avg_bkgrnd = (bkgrnd_subtracted[keys].loc['g', 11] + bkgrnd_subtracted[keys].loc['h', 11])/2
  #  bkgrnd_subtracted[keys] = bkgrnd_subtracted[keys] - avg_bkgrnd

In [ ]:
for key in raw_data.keys():
    for index, row in metadata[metadata['Plate#'] == key].iterrows():
        well = metadata['Location'][index]
        OD = raw_data[key].loc[well[0],well[1:]]
        metadata['OD'][index] = OD

In [ ]:
background = metadata[metadata['Type'] == 'Blank']['OD'].mean()
bkgrnd_subtracted = metadata.copy()
bkgrnd_subtracted['OD'] = bkgrnd_subtracted['OD'] - background

In [ ]:
def cv(x):
    return (x.std() / x.mean()) * 100

agg_data = bkgrnd_subtracted.groupby(by = ['ID']).agg({'OD': ['sum', cv]}).droplevel(level=0,axis=1)

In [ ]:
concentrations = pd.DataFrame(bkgrnd_subtracted).set_index('ID')
agg_data = agg_data.join(concentrations)

In [ ]:
standards = agg_data[agg_data['Type'] == 'Standard'].drop(['OD', 'Location'], axis = 1).drop_duplicates()
x = list(standards['Concentration'])
y = list(standards['sum'])
params, _ = curve_fit(log5pl,x,y)

In [ ]:
interpolated = agg_data.drop(['OD', 'Location'], axis = 1).drop_duplicates()
for index, row in interpolated.iterrows():
    inputs = np.insert(params, 0, row['sum'])
    conc = rev_log5pl(inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5])
    print(inputs)
    print(conc)
    interpolated['Concentration'][index] = conc

[-5.918       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-6.038       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-6.037       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-5.901       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-5.927       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-1.929       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
0.34314484899543635
[-5.936       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-6.035       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-6.036       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-5.845       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-5.981       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
nan
[-3.222       3.14922496  8.74578445  0.4144986  -5.76583336  4.80899804]
0.36090586611165126
[ 0.424       3.14922496  8.7457

In [ ]:
interpolated

,sum,cv,Plate#,Concentration,Type,Animal ID,Date,Timepoint
ID,,,,,,,,
12M273_D-7,-5.918,-0.286762,Plate#1,NaN,Sample,12M273,2022-07-15,D-7
12M273_D10,-6.038,-0.000000,Plate#1,NaN,Sample,12M273,2022-08-04,D10
12M273_D14,-6.037,-0.023426,Plate#1,NaN,Sample,12M273,2022-08-08,D14
12M273_D3,-5.901,-0.215691,Plate#1,NaN,Sample,12M273,2022-07-28,D3
12M273_D5,-5.927,-0.596513,Plate#1,NaN,Sample,12M273,2022-07-30,D5
12M273_D7,-1.929,-98.019882,Plate#1,0.343145,Sample,12M273,2022-08-01,D7
15P034_D-7,-5.936,-0.142946,Plate#1,NaN,Sample,15P034,2022-07-15,D-7
15P034_D10,-6.035,-0.023434,Plate#1,NaN,Sample,15P034,2022-08-04,D10
15P034_D14,-6.036,-0.000000,Plate#1,NaN,Sample,15P034,2022-08-08,D14


nan